# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

#### Analyis
Observed Trends
1.  
2.  
3.  

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
# Build dataframe using list cities generated in above steps 
weather_df = pd.DataFrame({"City" : cities,
                         "Lat" : "",
                         "Lng" : "",
                         "Max Temp" : "",
                         "Humidity" : "",
                         "Cloudiness" : "",
                         "Wind Speed" : "",
                         "Date" : ""})

In [ ]:
# https://openweathermap.org/current#name
base_url = "https://api.openweathermap.org/data/2.5/weather?"

print('Beginning Data Retrieval')     
print('-----------------------------')

# Loop through the number of cities
for x in range(len(cities)):
               
    city = cities[x]
    units = "imperial"
    url = f"{base_url}q={city}&appid={weather_api_key}&units={units}"
    
    # call API for each city and add data to dataframe if data is available
    try: 
        weather_data = requests.get(url).json()
    
        weather_df.loc[weather_df['City'] == city, 'Lat'] = weather_data['coord']['lat']
        weather_df.loc[weather_df['City'] == city, 'Lng'] = weather_data['coord']['lon']
        weather_df.loc[weather_df['City'] == city, 'Max Temp'] = weather_data['main']['temp_max']
        weather_df.loc[weather_df['City'] == city, 'Humidity'] = weather_data['main']['humidity']
        weather_df.loc[weather_df['City'] == city, 'Cloudiness'] = weather_data['clouds']['all']
        weather_df.loc[weather_df['City'] == city, 'Wind Speed'] = weather_data['wind']['speed']
        weather_df.loc[weather_df['City'] == city, 'Country'] = weather_data['sys']['country']
        weather_df.loc[weather_df['City'] == city, 'Date'] = weather_data['dt']
        
        print(f'Processing Record {x+1} | {city} ')
    except:
        print('City not found. Skipping...')

In [ ]:
# Check counts for each variable to see if there are NaN or missing values
weather_df.count()

In [ ]:
# Drop missing values
weather_df.dropna(inplace=True)

# Reset index to adjust for NaN values that were removed 
weather_df.reset_index(drop=True,inplace=True)

In [ ]:
# Failsafe to keep the number of cities under 600 for purposes of not testing API call limits
weather_df = weather_df[weather_df.index < 600]

In [ ]:
# Change datatypes for numeric data 
weather_df[['Lat','Lng','Max Temp','Wind Speed']] = weather_df[['Lat','Lng',
                                                                'Max Temp',
                                                                'Wind Speed']].astype(float)
weather_df[['Humidity','Cloudiness']] = weather_df[['Humidity','Cloudiness']].astype(int)

In [ ]:
weather_df.count()

RETURN HERE TO ADD EXPORT TO CSV

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
weather_df.head()

In [ ]:
weather_df.describe()

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
humid_cities = weather_df.loc[weather_df['Humidity'] > 100,:]
humid_cities

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".
clean_city_data = pd.DataFrame(weather_df).copy()

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs in the output_data folder
* Below are examples of what you should get but your results will be different.

## Latitude vs. Temperature Plot

In [ ]:
x_axis = clean_city_data['Lat']
y_axis = clean_city_data['Max Temp']

plt.xlabel('Latitude')
plt.ylabel('Max Temperature (F)')
plt.title('City Latitude vs. Max Temperature (04/10/22)')
plt.grid()
plt.scatter(x_axis,y_axis,edgecolors='black')

## Latitude vs. Humidity Plot

In [ ]:
x_axis = clean_city_data['Lat']
y_axis = clean_city_data['Humidity']

plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.title('City Latitude vs. Humidity (04/10/22)')
plt.grid()
plt.scatter(x_axis,y_axis,edgecolors='black')

## Latitude vs. Cloudiness Plot

In [ ]:
x_axis = clean_city_data['Lat']
y_axis = clean_city_data['Cloudiness']

plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
plt.title('City Latitude vs. Cloudiness (04/10/22)')
plt.grid()
plt.scatter(x_axis,y_axis,edgecolors='black')

## Latitude vs. Wind Speed Plot

In [ ]:
x_axis = clean_city_data['Lat']
y_axis = clean_city_data['Wind Speed']

plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
plt.title('City Latitude vs. Wind Speed (04/10/22)')
plt.grid()
plt.scatter(x_axis,y_axis,edgecolors='black')

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression